#### Modelling the sequential FPL team selection process as a Belief-State Markov Decision Process
- For the $i$-th gameweek, we define the following terms:
    - $M_i$ is set of matches in gameweek $i$.
    - $P_i$ is the set of players available for selection in gameweek $i$.
    - $A_i$ is the set of actions available in gameweek $i$, where $a \in A_i$ is a subset of $P_i$ and observes all team selection constraints.
    - $p_i \in P_i$ is associated with its FPL-designated position $pos(p_i)$ and price $pr(p_i)$.
    - $\tau_p \in \tau$ is a system of distributions representing player's performance/influence on the matchplay.
    - $O_i$ is the set of match observations in gameweek $i$
    - $o \in O_i$ includes both the result of the matches and the performance of the players in the selected team e.g. goals, assists, clean sheets, yellow cards, red cards, bonus points. The probability of each $o \in O_i$ is somehow dependent on the players' characteristics ($\tau$) i.e. a team with strong attackers is more likely to score goals, therefore, $P(o | \tau)$ is dependent on $\tau$.
    - $R(o, a_{prev}, a_{curr})$ is the reward function, which returns the points scored by the selected team $a_{curr}$, given the match observations $o$. The previous team $a_{prev}$ is also provided to penalize the agent for any player poor player transfers or transfers beyond the allowed number.

Markov's Decision Process (MDP) 
- A state $S_i$ would encapsulate
    - $M_{i,..., 38}$ - set of upcoming fixtures for that gameweek
    - $P_i$ - set of players available for selection
    - $o \in O_{i - 1}$ - the outcome of the previous gameweek
    - $\tau$ - the system of distributions representing players' abilities
- An action $A_i$ is the set of teams selectable in gameweek $i$
- $R$ is the corresponding reward function

Belief model ($\tau$):
- Represent uncertainty over players' abilities and generate samples $\tau$ from the distribution $Pr(\tau | b)$.
- Three distributions are used to model the players' abilities:
    - $\rho_p$ - a three-state categorical distribution representing the player's probability of starting a match, being substituted, or not playing at all i.e. (start, sub, unused).
    - $\omega_p$ - a Bernoulli/Binomial distribution over a single trial, representing the probability of a player scoring a goal given he was playiong at the time
    - $\psi_p$ - a Bernoulli distribution representing the probability of a player providing an assist given he was playing at the time
- Define prior distributions over the parameters of the above distributions and update them using the match observations $o$ to obtain new posterior distributions.
- Use simple closed-form equations e.g. Beta and Dirichlet conjugate priors to update the priors.
- Sample from these conjugate distributions to generate $\tau_p$.
- Define hyperparemeters uniformly across all players i.e. $$\omega_p \sim Beta(1, 1), \psi_p \sim Beta(1, 1),  \rho_p \sim Dirichlet(\frac{1}{4}, \frac{1}{4}, \frac{1}{4})$$
- Potential to use performance data from previous seasons to define priors
- Define 4 global multinomial distributions $S_{pos}$ - one for each position - to describe the distribution of minutes players who play the same position $pos$ are likely to play in a match, given they start the match.
- Player absence via injury/suspension or any other reson is modelled by setting the probability of starting and substituting to zero i.e. $Pr(\rho_p = start) \text{and} Pr(\rho_p = sub) = 0$.

In [ ]:
from utils.data_registry import DataRegistry
import pymc as pm
import numpy as np
import pandas as pd
import arviz as az

# There are 38 gameweeks in a season
GAMEWEEK_COUNT = 38

seasonal_gameweek_player_data = DataRegistry()

In [ ]:
def train_player_ability_priors():
    # Define player ability (only for players in the 2023/24 season) priors to reflect the occurrences of previous seasons i.e. 2016/17 - 2022/23
    player_ability_df = pd.DataFrame({
        "name": seasonal_gameweek_player_data.player_data["2023-24"]["first_name"] + " " + seasonal_gameweek_player_data.player_data["2023-24"]["second_name"],
        "ρ_β": [np.array([1/4, 1/4, 1/4])] * len(seasonal_gameweek_player_data.player_data["2023-24"]), # Dirichlet prior for ρ
        ("ω", "α"): np.ones(len(seasonal_gameweek_player_data.player_data["2023-24"])), # Beta prior for ω
        ("ω", "β"): np.ones(len(seasonal_gameweek_player_data.player_data["2023-24"])), # Beta prior for ω
        ("ψ", "α"): np.ones(len(seasonal_gameweek_player_data.player_data["2023-24"])), # Beta prior for ψ
        ("ψ", "β"): np.ones(len(seasonal_gameweek_player_data.player_data["2023-24"])) # Beta prior for ψ
    })
    player_priors = {
        player_name: {
            "ρ_β": np.array([1/4, 1/4, 1/4]),
            "ω_α": 1.0,
            "ω_β": 1.0,
            "ψ_α": 1.0,
            "ψ_β": 1.0
        }
        for player_name in player_ability_df["name"]
    }

    for season, gameweek_data_df in seasonal_gameweek_player_data.gameweek_data.items():
        for gameweek_count in range(1, GAMEWEEK_COUNT + 1):
            # Filter out players not present in the 2023/24 season
            gameweek_data_df = gameweek_data_df[gameweek_data_df["name"].isin(player_ability_df["name"])]
            grouped_data = gameweek_data_df.groupby(["name", "GW"])
            for (player_name, _), player_gameweek_data in grouped_data:
                priors = player_priors[player_name]
                with pm.Model() as sequential_player_ability_model:

                    # Priors for the current iteration
                    ρ = pm.Dirichlet('ρ', a=priors["ρ_β"])
                    ω = pm.Beta('ω', alpha=priors["ω_α"], beta=priors["ω_β"])
                    ψ = pm.Beta('ψ', alpha=priors["ψ_α"], beta=priors["ψ_β"])

                    # ρ Likelihood (multinomial) -> (played, subbed, not_used)
                    ρ_observed = np.zeros(3)
                    if player_gameweek_data["minutes"].iloc[0] == 0:
                        ρ_observed[2] = 1
                    else:
                        if player_gameweek_data["starts"].iloc[0] == 1:
                            ρ_observed[0] = 1
                        else:
                            ρ_observed[1] = 1
                    ρ_likelihood = pm.Multinomial(name="ρ_likelihood", n=np.sum(ρ_observed), p=ρ, observed=ρ_observed)
                    # ω Likelihood (binomial)
                    ω_observed = player_gameweek_data["goals_scored"].sum()
                    ω_likelihood = pm.Binomial(name="ω_observed", n=ω_observed, p=ω, observed=ω_observed)
                    # ψ Likelihood (binomial)
                    ψ_observed = player_gameweek_data["assists"].sum()
                    ψ_likelihood = pm.Binomial(name="ψ_observed", n=ψ_observed, p=ψ, observed=ψ_observed)

                    # Sample the posterior
                    with sequential_player_ability_model:
                        approx = pm.fit(n=1000, method="advi")
                        trace = approx.sample(1000)
                
                    # Update priors for the next iteration
                    posterior_means = az.summary(trace, var_names=["ρ", "ω", "ψ"])
                    priors["ρ_β"] = posterior_means.loc["ρ_β", "mean"][:3]
                    priors["ω_α"] = posterior_means.loc[:, 'mean'][3]
                    priors["ω_β"] = 1 - priors["ω_α"]
                    priors["ψ_α"] = posterior_means.loc[:, 'mean'][4]
                    priors["ψ_β"] = 1 - priors["ψ_α"]
                print(f"{player_name}")
                # Save intermediate results every 5 gameweeks
            if gameweek_count % 5 == 0:
                player_ability_df.to_csv(path_or_buf=f"./data/player_ability_results/player_ability_gw_{gameweek_count}.csv", index=False)

    # Update player_ability_df with the updated priors from player_priors
    for player_name, priors in player_priors.items():
        player_ability_df.loc[player_ability_df["name"] == player_name, "ρ_β"] = [priors["ρ_β"]]
        player_ability_df.loc[player_ability_df["name"] == player_name, ("ω", "α")] = priors["ω_α"]
        player_ability_df.loc[player_ability_df["name"] == player_name, ("ω", "β")] = priors["ω_β"]
        player_ability_df.loc[player_ability_df["name"] == player_name, ("ψ", "α")] = priors["ψ_α"]
        player_ability_df.loc[player_ability_df["name"] == player_name, ("ψ", "β")] = priors["ψ_β"]
    # Save player_ability_df as a csv file
    player_ability_df.to_csv(path_or_buf="./data/player_ability.csv", index=False)
    return 



In [ ]:
def train_player_position_minutes_priors():
    # Global multinomial distributions for players' minutes played at each position
    player_positions = seasonal_gameweek_player_data.player_data["2023-24"]["element_type"].unique()
    player_position_minutes_df = pd.DataFrame({
        "position": player_positions,
        "minutes": [np.zeros(91) for _ in range(len(player_positions))] # Dirichlet prior for minutes at each position
    })
    seasons_with_player_position_data = {"2020-21", "2021-22", "2022-23"}
    # Use a dictionary for faster indexing in the for loop
    position_priors = {
        position: np.ones(91)
        for position in player_positions
    }
    for season, gameweek_data_df in seasonal_gameweek_player_data.gameweek_data.items():
        if season in seasons_with_player_position_data:
            group_data = gameweek_data_df.groupby(["position"])
            for position, group_df in group_data:
                # Aggregate observed minutes played for the position
                observed_minutes = np.zeros(91)
                for minutes_played in group_df["minutes"]:
                    if 0 <= minutes_played <= 90:
                        observed_minutes[int(minutes_played)] += 1
                with pm.Model() as player_position_minutes_played_model:
                    # Dirichlet prior for minutes played at each position
                    prior = pm.Dirichlet(f"prior_{position}", a=position_priors[position], shape=91)
                    likelihood = pm.Multinomial(
                        name=f"likelihood_{position}", 
                        n=np.sum(observed_minutes),
                        p=prior,
                        observed=observed_minutes)
                    # Sample the posterior
                    approx = pm.fit(n=1000, method="advi")
                    trace = approx.sample(1000)
                    # Update priors for the next iteration
                    posterior_means = az.summary(trace, var_names=[f"prior_{position}"]).loc[:, "mean"].values
                    position_priors[position] = posterior_means
    # Transfer priors to player_position_minutes_df
    for position, priors in position_priors.items():
        player_position_minutes_df.loc[player_position_minutes_df["position"] == position, "minutes"] == [priors]
    # Save player_position_minutes_df as a csv file
    player_position_minutes_df.to_csv(path_or_buf="./data/player_position_minutes.csv", index=False)
    return 

['FWD' 'DEF' 'MID' 'GK']


In [ ]:
train_player_ability_priors()